In [ ]:
import random
import numbers
import os
import os.path
import numpy as np
import struct
import math

import torch
import torchvision
import matplotlib.pyplot as plt
import h5py
import json

from util import som

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib qt5 


In [ ]:
# for train and val
def som_saver_shrec2016(root, rows, cols, gpu_ids, output_root):
    som_builder = som.SOM(rows, cols, 3, gpu_ids)
    
    folder_list = os.listdir(root)
    for i, folder in enumerate(folder_list):
        file_list = os.listdir(os.path.join(root, folder))
        for j, file in enumerate(file_list):
            if file[-3:] == 'txt':
                data = np.loadtxt(os.path.join(root, folder, file))
                pc_np = data[:, 0:3]
                sn_np = data[:, 3:6]
                
                pc_np_sampled = pc_np[np.random.choice(pc_np.shape[0], 4096, replace=False), :]
                pc = torch.from_numpy(pc_np_sampled.transpose().astype(np.float32)).cuda()  # 3xN tensor
                som_builder.optimize(pc)
                som_node_np = som_builder.node.cpu().numpy().transpose().astype(np.float32)  # node_numx3

                npz_file = os.path.join(output_root, file[0:-4]+'.npz')
                np.savez(npz_file, pc=pc_np, sn=sn_np, som_node=som_node_np)

                if j%100==0:
                    print('%s, %s' % (folder, file))

#                 print(pc_np.shape)
#                 print(som_node_np.shape)

#                 x_np = pc_np
#                 node_np = som_node_np
#                 fig = plt.figure()
#                 ax = Axes3D(fig)
#                 ax.scatter(x_np[:,0].tolist(), x_np[:,1].tolist(), x_np[:,2].tolist(), s=1)
#                 ax.scatter(node_np[:,0].tolist(), node_np[:,1].tolist(), node_np[:,2].tolist(), s=6, c='r')
#                 plt.show()

#                 if j>10:
#                     break
#         break

In [ ]:
rows, cols = 8, 8
som_saver_shrec2016('/ssd/dataset/SHREC2016/obj_txt/train', rows, cols, True, '/ssd/dataset/SHREC2016/%dx%d/train'%(rows, cols))
som_saver_shrec2016('/ssd/dataset/SHREC2016/obj_txt/val', rows, cols, True, '/ssd/dataset/SHREC2016/%dx%d/val'%(rows, cols))

In [ ]:
# for test set
def som_saver_shrec2016(root, rows, cols, gpu_ids, output_root):
    som_builder = som.SOM(rows, cols, 3, gpu_ids)

    file_list = os.listdir(root)
    for j, file in enumerate(file_list):
        if file[-3:] == 'txt':
            data = np.loadtxt(os.path.join(root, file))
            pc_np = data[:, 0:3]
            sn_np = data[:, 3:6]

            pc_np_sampled = pc_np[np.random.choice(pc_np.shape[0], 4096, replace=False), :]
            pc = torch.from_numpy(pc_np_sampled.transpose().astype(np.float32)).cuda()  # 3xN tensor
            som_builder.optimize(pc)
            som_node_np = som_builder.node.cpu().numpy().transpose().astype(np.float32)  # node_numx3

            npz_file = os.path.join(output_root, file[0:-4]+'.npz')
            np.savez(npz_file, pc=pc_np, sn=sn_np, som_node=som_node_np)

            if j%100==0:
                print('%s' % (file))

#                 print(pc_np.shape)
#                 print(som_node_np.shape)

#                 x_np = pc_np
#                 node_np = som_node_np
#                 fig = plt.figure()
#                 ax = Axes3D(fig)
#                 ax.scatter(x_np[:,0].tolist(), x_np[:,1].tolist(), x_np[:,2].tolist(), s=1)
#                 ax.scatter(node_np[:,0].tolist(), node_np[:,1].tolist(), node_np[:,2].tolist(), s=6, c='r')
#                 plt.show()

#                 if j>10:
#                     break
#         break

In [ ]:
rows, cols = 8, 8
som_saver_shrec2016('/ssd/dataset/SHREC2016/obj_txt/test_allinone', rows, cols, True, '/ssd/dataset/SHREC2016/%dx%d/test'%(rows, cols))

In [ ]:
file = '/ssd/dataset/SHREC2016/8x8/train/model_013435.npz'
data = np.load(file)
pc_np = data['pc']
sn_np = data['sn']
som_node_np = data['som_node']

print(pc_np)
print(sn_np)
print(som_node_np)

x_np = pc_np
node_np = som_node_np
fig = plt.figure()
ax = Axes3D(fig)
ax.scatter(x_np[:,0].tolist(), x_np[:,1].tolist(), x_np[:,2].tolist(), s=1)
ax.scatter(node_np[:,0].tolist(), node_np[:,1].tolist(), node_np[:,2].tolist(), s=6, c='r')
plt.show()

In [ ]:
file_list = os.listdir('/ssd/dataset/SHREC2016/8x8/test')
file_list.sort()
f = open('test.txt', 'w')
for file in file_list:
    f.write('%s\n' % file[6:-4])
f.close()